In [1]:
import numpy as np
from collections import defaultdict

"""
YOU MUST!!!
Read all the lines of the code provided to you and understand what it does!
"""


class DecisionNode(object):
    """
    README
    DecisionNode is a building block for Decision Trees.
    DecisionNode is a python class representing a  node in our decision tree
    node = DecisionNode()  is a simple usecase for the class
    you can also initialize the class like this:
    node = DecisionNode(column = 3, value = "Car")
    In python, when you initialize a class like this, its __init__ method is called 
    with the given arguments. __init__() creates a new object of the class type, and initializes its 
    instance attributes/variables.
    In python the first argument of any method in a class is 'self'
    Self points to the object which it is called from and corresponds to 'this' from Java

    """

    def __init__(self,
                 column=None,
                 value=None,
                 false_branch=None,
                 true_branch=None,
                 current_results=None,
                 is_leaf=False,
                 results=None):
        self.column = column
        self.value = value
        self.false_branch = false_branch
        self.true_branch = true_branch
        self.current_results = current_results
        self.is_leaf = is_leaf
        self.results=results

def dict_of_values(data):
    """
    param data: a 2D Python list representing the data. Last column of data is Y.
    return: returns a python dictionary showing how many times each value appears in Y

    for example 
    data = [[1,'yes'],[1,'no'],[1,'yes'],[1,'yes']]
    dict_of_values(data)
    should return {'yes' : 3, 'no' :1}
    """
    results = defaultdict(int)
    for row in data:
        r = row[-1]
        results[r] += 1
    return dict(results)


def divide_data(data, feature_column, feature_val):
    """
    this function dakes the data and divides it in two parts by a line. A line
    is defined by the feature we are considering (feature_column) and the target 
    value. The function returns a tuple (data1, data2) which are the desired parts of the data.
    For int or float types of the value, data1 have all the data with values >= feature_val
    in the corresponding column and data2 should have rest.
    For string types, data1 should have all data with values == feature val and data2 should 
    have the rest.

    param data: a 2D Python list representing the data. Last column of data is Y.
    param feature_column: an integer index of the feature/column.
    param feature_val: can be int, float, or string
    return: a tuple of two 2D python lists
    """
    if type(feature_val) == int or type(feature_val) == float:
        data1=[e for e in data if e[feature_column]>=feature_val]
        data2=[e for e in data if e[feature_column]<feature_val]
        pass
    else:
        data1=[e for e in data if e[feature_column]==feature_val]
        data2=[e for e in data if e[feature_column]!=feature_val]
        pass
    return data1, data2


def gini_impurity(data1, data2):

    """
    Given two 2D lists of compute their gini_impurity index. 
    Remember that last column of the data lists is the Y
    Lets assume y1 is y of data1 and y2 is y of data2.
    gini_impurity shows how diverse the values in y1 and y2 are.
    gini impurity is given by 

    N1*sum(p_k1 * (1-p_k1)) + N2*sum(p_k2 * (1-p_k2))

    where N1 is number of points in data1
    p_k1 is fraction of points that have y value of k in data1
    same for N2 and p_k2


    param data1: A 2D python list
    param data2: A 2D python list
    return: a number - gini_impurity 
    """

    results1 = defaultdict(int)
    a1=[]
    l1=0
    for row in data1:
        r = row[len(row) - 1]
        if results1[r]==0:
            a1.append(r)
        results1[r] += 1
        l1+=1
    results2 = defaultdict(int)
    a2=[]
    l2=0
    for row in data2:
        r = row[len(row) - 1]
        if results2[r]==0:
            a2.append(r)
        results2[r] += 1
        l2+=1
    s1=0
    for i in a1:
        s1+=results1[i]*(1-results1[i]/l1)
    s2=0
    for i in a2:
        s2+=results2[i]*(1-results2[i]/l2)
    return s1 + s2


def build_tree(data, current_depth=0, max_depth=1e10):
    """
    build_tree is a recursive function.
    What it does in the general case is:
    1: find the best feature and value of the feature to divide the data into
    two parts
    2: divide data into two parts with best feature, say data1 and data2
        recursively call build_tree on data1 and data2. this should give as two 
        trees say t1 and t2. Then the resulting tree should be 
        DecisionNode(...... true_branch=t1, false_branch=t2) 


    In case all the points in the data have same Y we should not split any more, and return that node
    For this function we will give you some of the code so its not too hard for you ;)
    
    param data: param data: A 2D python list
    param current_depth: an integer. This is used if we want to limit the numbr of layers in the
        tree
    param max_depth: an integer - the maximal depth of the representing
    return: an object of class DecisionNode

    """
    if(current_depth == max_depth):
        return DecisionNode(current_results=dict_of_values(data),is_leaf=True)

    if(len(dict_of_values(data)) == 1):
        return DecisionNode(current_results=dict_of_values(data), is_leaf=True)

    #This calculates gini number for the data before dividing 
    self_gini = gini_impurity(data, [])

    #Below are the attributes of the best division that you need to find. 
    #You need to update these when you find a division which is better
    best_gini = 1e10
    best_column = None
    best_value = None
    #best_split is tuple (data1,data2) which shows the two datas for the best divison so far
    best_split = None
    for i in data:
        for j in range(len(i)-1):
            d1,d2=divide_data(data,j,i[j])
            gini=gini_impurity(d1,d2)
            if gini<best_gini:
                best_gini=gini
                best_column=j
                best_value=i[j]
                best_split=(d1,d2)
    #You need to find the best feature to divide the data
    #For each feature and each possible value of the feature compute the 
    # gini number for that division. You need to find the feature that minimizes
    # gini number. Remember that last column of data is Y
    # Think how you can use the divide_data and gini_impurity functions you wrote 
    # above
    

    #if best_gini is no improvement from self_gini, we stop and return a node.
    if abs(self_gini - best_gini) < 1e-10:
        return DecisionNode(current_results=dict_of_values(data), is_leaf=True)
    else:
        #recursively call build tree, construct the correct return argument and return
        return DecisionNode( #<---- FIX THIS
            column=best_column,
            value=best_value,
            is_leaf=False,
            false_branch=build_tree(best_split[1],current_depth+1,max_depth),
            true_branch=build_tree(best_split[0],current_depth+1,max_depth),
            current_results=dict_of_values(data)
        )


def print_tree(tree, indent=''):
    # Is this a leaf node?
    if tree.is_leaf:
        print(str(tree.current_results))
    else:
        # Print the criteria
        #         print (indent+'Current Results: ' + str(tree.current_results))
        print('Column ' + str(tree.column) + ' : ' + str(tree.value) + '? ')

        # Print the branches
        print(indent + 'True->')
        print_tree(tree.true_branch, indent + '  ')
        print(indent + 'False->')
        print_tree(tree.false_branch, indent + '  ')

In [2]:
class DecisionTree(object):
    """
    DecisionTree class, that represents one Decision Tree

    :param max_tree_depth: maximum depth for this tree.
    """
    def __init__(self, max_tree_depth):
        self.max_depth = max_tree_depth

    def fit(self, X, Y):
        """
        :param X: 2 dimensional python list or numpy 2 dimensional array
        :param Y: 1 dimensional python list or numpy 1 dimensional array
        """
        if not isinstance(X,list):
            X = X.tolist()
        if not isinstance(Y,list):
            Y= Y.tolist()
        data=[x+y for x,y in zip(X,Y)]
        self.trees=build_tree(data,0,self.max_depth)

    def predict(self, X):
        """
        :param X: 2 dimensional python list or numpy 2 dimensional array
        :return: Y - 1 dimension python list with labels
        """
        Y=[]
        for x in X:
            tree=self.trees
            while not tree.is_leaf:
                if x[tree.column] >= tree.value:
                    tree=tree.true_branch
                else:
                    tree=tree.false_branch
            Y.append(max(tree.current_results,key=tree.current_results.get))
        return Y

In [3]:

class RandomForest(object):
    """
    RandomForest a class, that represents Random Forests.

    :param num_trees: Number of trees in the random forest
    :param max_tree_depth: maximum depth for each of the trees in the forest.
    :param ratio_per_tree: ratio of points to use to train each of
        the trees.
    """
    def __init__(self, num_trees, max_tree_depth, ratio_per_tree=0.5):
        self.num_trees = num_trees
        self.max_tree_depth = max_tree_depth
        self.ratio_per_tree=ratio_per_tree
        self.trees = None

    def fit(self, X, Y):
        """
        :param X: 2 dimensional python list or numpy 2 dimensional array
        :param Y: 1 dimensional python list or numpy 1 dimensional array
        """
        self.trees = []
        for _ in range(self.num_trees):
            indices=np.arange(Y.shape[0])
            np.random.shuffle(indices)
            X=X[indices]
            Y=Y[indices]
            test_len=int(self.ratio_per_tree*len(X))
            X_train=X[:test_len]
            Y_train=Y[:test_len]
            tree=DecisionTree(self.max_tree_depth)
            tree.fit(X_train,Y_train)
            self.trees.append(tree)

    def predict(self, X):
        """
        :param X: 2 dimensional python list or numpy 2 dimensional array
        :return: (Y, conf), tuple with Y being 1 dimension python
        list with labels, and conf being 1 dimensional list with
        confidences for each of the labels.
        """
        predicts=[tree.predict(X) for tree in self.trees]
        predicts=[[predicts[i][j] for i in range(len(predicts))] for j in range(len(predicts[0]))]
        Y=[]
        conf=[]
        for r in predicts:
            results = defaultdict(np.float64)
            count=0
            for key in r:
                results[key]+=1
                count+=1
            Y.append(max(results,key=results.get))
            conf.append(results[Y[-1]]/count)
        return (Y, conf)


In [4]:

def sigmoid(s):
    return 1 - 1 / (1 + np.exp(s)) if abs(s) < 10 else 0 if s < 0 else 1


def normalized_gradient(X, Y,beta,l):
    """
    :param X: data matrix (2 dimensional np.array)
    :param Y: response variables (1 dimensional np.array)
    :param beta: value of beta (1 dimensional np.array)
    :param l: regularization parameter lambda
    :return: normalized gradient, i.e. gradient normalized according to data
    """
    gr = l * beta
    s = 0
    for i in range(X.shape[0]):
        s += X[i] * Y[i] * (1 - sigmoid(Y[i] * beta.T.dot(X[i])))
    gr -= s
    s = 0
    for i in range(X.shape[0]):
        s += Y[i] * (1 - sigmoid(Y[i] * beta.T.dot(X[i])))
    gr[0] -= s
    return gr / X.shape[0]


def gradient_descent(xx, yy,l,epsilon,max_steps,step_size,beta):
    """
    Implement gradient descent using full value of the gradient.
    :param X: data matrix (2 dimensional np.array)
    :param Y: response variables (1 dimensional np.array)
    :param l: regularization parameter lambda
    :param epsilon: approximation strength
    :param max_steps: maximum number of iterations before algorithm will
        terminate.
    :return: value of beta (1 dimensional np.array)
    """
    X = xx.copy()
    Y = yy.copy()
    gr = np.zeros(X.shape[1])

    v = np.var(X, axis=0, dtype=float) ** 0.5
    m = np.mean(X, axis=0, dtype=float)
    for i in range(1, X.shape[1]):
        for j in range(X.shape[0]):
            if v[i] != 0:
                X[j][i] -= m[i]
                X[j][i] /= v[i]
    l = np.array([l / v[_] ** 2 if v[_] != 0 else 0 for _ in range(X.shape[1])])

    for s in range(max_steps):
        old = beta.copy()
        gr = step_size * normalized_gradient(X, Y,beta,l)
        beta -= gr
        if ((beta - old) ** 2).sum() / ((beta ** 2).sum()) < epsilon ** 2:
            break
        pass
    for i in range(X.shape[1]):
        if i == 0:
            beta[0] = beta[0] - np.sum(
                np.array([m[i] * beta[i] / v[i] if v[i] != 0 else 0 for i in range(1, m.shape[0])]))
        else:
            beta[i] = beta[i] / v[i] if v[i] != 0 else 0
    return beta


class logistic(object):
    def __init__(self,epsilon=1e-6, l=1, step_size=1e-4, max_steps=1000):
        self.epsilon=epsilon
        self.step_size=step_size
        self.max_steps=max_steps
        self.l=l
    def fit(self,xx,yy):
        """
        :param xx: 2 dimensional python list or numpy 2 dimensional array
        :param yy: 1 dimensional python list or numpy 1 dimensional array
        """
        self.beta=np.random.normal(0,1,xx.shape[1])
        self.beta=gradient_descent(xx,yy,epsilon=self.epsilon,l=self.l,step_size=self.step_size,max_steps=self.max_steps,beta=self.beta)
    def predict(self,X):
        """
        :param X: 2 dimensional python list or numpy 2 dimensional array
        :return: (Y, conf), tuple with Y being 1 dimension python
        list with labels, and conf being 1 dimensional list with
        confidences for each of the labels.
        """
        return [1 if self.beta.dot(x)>0 else 0 for x in X]


In [5]:
import matplotlib.pyplot as plt


def accuracy_score(Y_true, Y_predict):
    c=0.
    for i in range(len(Y_true)):
        if Y_true[i]==Y_predict[i]:
            c+=1
    return c/len(Y_true)

def evaluate_performance():
    '''
    Evaluate the performance of decision trees and logistic regression,
    average over 1,000 trials of 10-fold cross validation

    Return:
      a matrix giving the performance that will contain the following entries:
      stats[0,0] = mean accuracy of decision tree
      stats[0,1] = std deviation of decision tree accuracy
      stats[1,0] = mean accuracy of logistic regression
      stats[1,1] = std deviation of logistic regression accuracy

    ** Note that your implementation must follow this API**
    '''

    # Load Data
    stats = np.zeros((3, 2))
    filename = 'SPECTF.dat'
    data = np.loadtxt(filename, delimiter=',')
    X = data[:, 1:]
    y = np.array([data[:, 0]]).T
    n, d = X.shape
    num_folds=10
    num_trials=50
    all_accuracies=[]
    rf_all_accuracies=[]
    l_all_accuracies=[]
    for trial in range(num_trials):
        idx = np.arange(n)
        np.random.seed(13)
        np.random.shuffle(idx)
        X = X[idx]
        y = y[idx]

        Xtrain = X[:n-n//num_folds, :]  # train on first 100 instances
        Xtest = X[n-n//num_folds:, :]
        ytrain = y[:n-n//num_folds, :]  # test on remaining instances
        ytest = y[n-n//num_folds:, :]

        # train the decision tree
        classifier = DecisionTree(5)
        classifier.fit(Xtrain, ytrain)
        # output predictions on the remaining data
        y_pred = classifier.predict(Xtest)
        accuracy = accuracy_score(ytest, y_pred)
        all_accuracies.append(accuracy)

        rf_classifier=RandomForest(5,100,0.3)
        rf_classifier.fit(Xtrain,ytrain)
        rf_y_pred=rf_classifier.predict(Xtest)[0]
        rf_accuracy=accuracy_score(ytest,rf_y_pred)
        rf_all_accuracies.append(rf_accuracy)

        l_classifier=logistic()
        l_classifier.fit(Xtest,ytest)

        l_y_pred=l_classifier.predict(Xtest)
        l_accuracy=accuracy_score(ytest,l_y_pred)
        l_all_accuracies.append(l_accuracy)
        print('iter num :{0}'.format(trial))
        
        meanDecisionTreeAccuracy = np.mean(all_accuracies)
        stddevDecisionTreeAccuracy = np.std(all_accuracies)
        meanLogisticRegressionAccuracy = np.mean(l_all_accuracies)
        stddevLogisticRegressionAccuracy = np.std(l_all_accuracies)
        meanRandomForestAccuracy = np.mean(rf_all_accuracies)
        stddevRandomForestAccuracy = np.std(rf_all_accuracies)

    
        stats[0, 0] = meanDecisionTreeAccuracy
        stats[0, 1] = stddevDecisionTreeAccuracy
        stats[1, 0] = meanRandomForestAccuracy
        stats[1, 1] = stddevRandomForestAccuracy
        stats[2, 0] = meanLogisticRegressionAccuracy
        stats[2, 1] = stddevLogisticRegressionAccuracy
        
        
        print ("\tDecision Tree Accuracy = ", stats[0, 0], " (", stats[0, 1], ")" )
        print ("\tRandom Forest Tree Accuracy = ", stats[1, 0], " (", stats[1, 1], ")" )
        print ("\tLogistic Reg. Accuracy = ", stats[2, 0], " (", stats[2, 1], ")")

    # compute the training accuracy of the model
    meanDecisionTreeAccuracy = np.mean(all_accuracies)

    stddevDecisionTreeAccuracy = np.std(all_accuracies)
    meanLogisticRegressionAccuracy = np.mean(l_all_accuracies)
    stddevLogisticRegressionAccuracy = np.std(l_all_accuracies)
    meanRandomForestAccuracy = np.mean(rf_all_accuracies)
    stddevRandomForestAccuracy = np.std(rf_all_accuracies)

    # make certain that the return value matches the API specification
    stats = np.zeros((3, 2))
    stats[0, 0] = meanDecisionTreeAccuracy
    stats[0, 1] = stddevDecisionTreeAccuracy
    stats[1, 0] = meanRandomForestAccuracy
    stats[1, 1] = stddevRandomForestAccuracy
    stats[2, 0] = meanLogisticRegressionAccuracy
    stats[2, 1] = stddevLogisticRegressionAccuracy
    return stats


# Do not modify from HERE...
if __name__ == "__main__":
    stats = evaluate_performance()
    print ("Decision Tree Accuracy = ", stats[0, 0], " (", stats[0, 1], ")" )
    print ("Random Forest Tree Accuracy = ", stats[1, 0], " (", stats[1, 1], ")" )
    print ("Logistic Reg. Accuracy = ", stats[2, 0], " (", stats[2, 1], ")")
# ...to HERE.


iter num :0
	Decision Tree Accuracy =  0.769230769231  ( 0.0 )
	Random Forest Tree Accuracy =  0.769230769231  ( 0.0 )
	Logistic Reg. Accuracy =  0.730769230769  ( 0.0 )
iter num :1
	Decision Tree Accuracy =  0.769230769231  ( 0.0 )
	Random Forest Tree Accuracy =  0.807692307692  ( 0.0384615384615 )
	Logistic Reg. Accuracy =  0.730769230769  ( 0.0 )
iter num :2
	Decision Tree Accuracy =  0.820512820513  ( 0.0725237724294 )
	Random Forest Tree Accuracy =  0.820512820513  ( 0.0362618862147 )
	Logistic Reg. Accuracy =  0.794871794872  ( 0.0906547155367 )
iter num :3
	Decision Tree Accuracy =  0.817307692308  ( 0.0630522935029 )
	Random Forest Tree Accuracy =  0.798076923077  ( 0.0499630040645 )
	Logistic Reg. Accuracy =  0.788461538462  ( 0.0792904928003 )
iter num :4
	Decision Tree Accuracy =  0.807692307692  ( 0.0595843591724 )
	Random Forest Tree Accuracy =  0.8  ( 0.0448534761142 )
	Logistic Reg. Accuracy =  0.784615384615  ( 0.0713355268884 )
iter num :5
	Decision Tree Accuracy =  0.

In [6]:
filename = 'SPECTF.dat'
data = np.loadtxt(filename, delimiter=',')
X = data[:, 1:]
y = np.array([data[:, 0]]).T
n, d = X.shape
num_folds=10
    


In [7]:
idx = np.arange(n)
np.random.seed(10)
np.random.shuffle(idx)
X = X[idx]
y = y[idx]

Xtrain = X[:n-n//num_folds, :]  # train on first 100 instances
Xtest = X[n-n//num_folds:, :]
ytrain = y[:n-n//num_folds, :]  # test on remaining instances
ytest = y[n-n//num_folds:, :]

rf_classifier=RandomForest(15,100,0.3)
rf_classifier.fit(Xtrain,ytrain)

l_classifier=logistic()
l_classifier.fit(Xtest,ytest)
        

In [30]:
idx=-9
print("X={0} \nY={1}".format(X[idx],y[idx]))

X=[ 58.  63.  80.  71.  76.  70.  70.  71.  64.  63.  74.  78.  77.  75.  62.
  61.  62.  56.  71.  52.  82.  71.  84.  85.  71.  71.  57.  47.  42.  39.
  70.  70.  50.  70.  50.  46.  58.  60.  76.  73.  82.  77.  65.  66.] 
Y=[ 0.]


In [31]:
rf_classifier.predict([X[idx]])

([0.0], [0.66666666666666663])

In [32]:
l_classifier.predict([X[idx]])

[1]